# GACSE Workshop

In [1]:
from __future__ import print_function

In [2]:
import sys
sys.path.append('../build/')

In [33]:
%pylab inline
np.set_printoptions(precision=4, suppress=True)

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [4]:
import versor as vsr

In [5]:
from motor_estimation import MotorEstimationSolver

## Ground truth motor

In [6]:
motor = vsr.Vec(1,1,1).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
print(motor)

Mot: [ 0.87 0 -0.5 0 -0.68 -0.43 -0.18 -0.25 ]


## Motor Estimation from Points

### Dataset generation

In [7]:
n_points = 10
sigma = 0.09
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() 
            for i in range(n_points)]
points_b = [point.spin(motor) for point in points_a]
points_b_noisy = [vsr.Vec(*(np.array(point)[:3] 
                            + sigma * np.random.randn(3))).null() 
                  for point in points_b]

### Estimate motor

In [8]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(points_a, points_b):
    mes.add_point_correspondences_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = 'DENSE_QR'
(estimated_motor, summary) = mes.solve()
print(summary['brief_report'])

Ceres Solver Report: Iterations: 5, Initial cost: 1.612012e+01, Final cost: 4.681787e-19, Termination: CONVERGENCE


#### Check solution

In [9]:
print(np.array(motor))
print(np.array(estimated_motor))
print(np.allclose(np.array(motor), np.array(estimated_motor)))

[ 0.87  0.   -0.5   0.   -0.68 -0.43 -0.18 -0.25]
[ 0.87 -0.   -0.5   0.   -0.68 -0.43 -0.18 -0.25]
True


## Motor Estimation from Lines

### Dataset generation

In [10]:
lines_a = [vsr.Dll(vsr.Vec(*np.random.normal(0.0,0.8,3)),
                   vsr.Vec(*np.random.normal(0.0,0.8,3)))
          for i in range(10)]
lines_b = [line.spin(motor) for line in lines_a]

### Estimate motor using 6 residuals

$$ \arg \min_M \frac{1}{2} \sum_{i=1}^n \left| M \varLambda_{b_i} \tilde{M} - \varLambda_{a_i}  \right| $$

In [11]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(lines_a, lines_b):
    mes.add_line_correspondences_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = "DENSE_QR"
estimated_motor, summary = mes.solve()
print(summary['brief_report'])

Ceres Solver Report: Iterations: 6, Initial cost: 2.855485e+01, Final cost: 2.506454e-25, Termination: CONVERGENCE


#### Check solution

In [12]:
print(np.array(motor))
print(np.array(estimated_motor))
print(np.allclose(np.array(motor), np.array(estimated_motor)))

[ 0.87  0.   -0.5   0.   -0.68 -0.43 -0.18 -0.25]
[ 0.87  0.   -0.5   0.   -0.68 -0.43 -0.18 -0.25]
True


### Estimate motor using 4 residuals

In [13]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(lines_a, lines_b):
    mes.add_line_angle_distance_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = "DENSE_QR"
estimated_motor, summary = mes.solve()
print(summary['brief_report'])

Ceres Solver Report: Iterations: 7, Initial cost: 2.850646e+01, Final cost: 1.250001e+00, Termination: CONVERGENCE


#### Check solution

In [14]:
print(np.array(motor))
print(np.array(estimated_motor))
print(np.allclose(np.array(motor), np.array(estimated_motor)))
print(np.allclose(np.array(motor), np.array(estimated_motor), 
                  rtol=1e-05, atol=1e-03))

[ 0.87  0.   -0.5   0.   -0.68 -0.43 -0.18 -0.25]
[ 0.87  0.   -0.5   0.   -0.68 -0.43 -0.18 -0.25]
False
True


### Estimate motor using 2 residuals

In [15]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(lines_a, lines_b):
    mes.add_line_angle_distance_norm_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = "DENSE_QR"
estimated_motor, summary = mes.solve()
print(summary['brief_report'])

Ceres Solver Report: Iterations: 14, Initial cost: 2.850646e+01, Final cost: 1.250001e+00, Termination: CONVERGENCE


#### Check solution

In [16]:
print(np.array(motor))
print(np.array(estimated_motor))
print(np.allclose(np.array(motor), np.array(estimated_motor)))
print(np.allclose(np.array(motor), np.array(estimated_motor), 
                  rtol=1e-05, atol=1e-03))

[ 0.87  0.   -0.5   0.   -0.68 -0.43 -0.18 -0.25]
[ 0.87  0.   -0.5  -0.   -0.68 -0.43 -0.18 -0.25]
False
True


### Hand Eye Calibration

### Set up motors

In [17]:
M_object_in_world = vsr.Vec(1.0,1.0,0.1).trs() * vsr.Biv(pi/4.0, 0.0,0.0).exp()
M_eye_in_hand = vsr.Vec(0.1, 0.2, 0.3).trs() * vsr.Biv(0.0,1.0,0.0).exp()
M_eye_in_hand_initial = vsr.Vec(0.11, 0.24, 0.36).trs() * (vsr.Biv(1.0,1.0,0.0).unit() * (np.pi/5.)).exp()

### Dataset generation

In [18]:
def hand_eye_pose_pairs(M_object_in_world, M_eye_in_hand, n):
    pose_pairs = [
            (M_hand_in_world, 
             (M_eye_in_hand.rev() * M_hand_in_world.rev() * M_object_in_world))
            for M_hand_in_world in [vsr.Vec(*np.random.rand(3)).trs() *
                                    vsr.Rot(vsr.Biv(*np.random.rand(3)).unit() *
                                            np.random.rand() * np.pi)
                                    for i in range(n)]]

    As = [pose_pair[0] for pose_pair in pose_pairs]
    Bs = [pose_pair[1] for pose_pair in pose_pairs]

    LAs = []
    LBs = []
    for i in range(n):
        for j in range(i+1,n):
            LAs.append(((As[j].rev() * As[i]).log() * 0.5).unit())
            LBs.append(((Bs[j] * Bs[i].rev()).log() * 0.5).unit())
    return LAs, LBs

In [19]:
n_lines = 10
lines_a, lines_b = hand_eye_pose_pairs(M_object_in_world, M_eye_in_hand, n_lines)

### Estimate hand-eye motor 

In [131]:
mes = MotorEstimationSolver(M_eye_in_hand_initial)
for a, b in zip(lines_a, lines_b):
    mes.add_line_angle_distance_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = "DENSE_QR"
estimated_motor, summary = mes.solve()
print(summary['full_report'])


Solver Summary (v 1.12.0-eigen-(3.2.7)-lapack-suitesparse-(4.4.4)-cxsparse-(3.1.4)-openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                            20                       20
Residual                                   80                       80

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                     1                        1
Linear solver threads                       1                        1

Cost:
Initial                          4.791545e+02
Fin

#### Check solution

In [132]:
print(np.array(M_eye_in_hand))
print(np.array(estimated_motor))
print(np.allclose(np.array(M_eye_in_hand), np.array(estimated_motor)))

[ 0.5403  0.     -0.8415  0.     -0.1532 -0.054  -0.039  -0.0841]
[-0.2754 -0.0997 -0.757  -0.5841 -0.0562 -0.0528  0.3195  0.1418]
False


### Real Experiment

In [22]:
import cPickle as pickle
import math3d as m3d

#### Dataset generation

In [23]:
rob_pose_list = pickle.load(open('pose_list.dump'))
obj_pose_list = pickle.load(open('obj_pose_list.dump'))
a_trfs = pickle.load(open('as.dump'))
b_trfs = pickle.load(open('bs.dump'))

In [262]:
rob_motors = [vsr.Vec(*pos).trs() * vsr.Rot(vsr.Vec(*axis).unduale() * angle / 2.)
              for pos, (axis, angle) in [(m3d.Transform(pose).pos, 
                                          m3d.Transform(pose).orient.axis_angle)
                                         for pose in rob_pose_list]]
obj_motors = [vsr.Vec(*pos).trs() * vsr.Rot(vsr.Vec(*axis).unduale() * angle / 2.)
              for pos, (axis, angle) in [(m3d.Transform(pose).pos, 
                                          m3d.Transform(pose).orient.axis_angle)
                                         for pose in obj_pose_list]]

In [ ]:
# n_lines = len(rob_motors)
# lines_a = []
# lines_b = []
# for i in range(n_lines):
#     for j in range(i+1,n_lines):
#         lines_a.append(((rob_motors[j].rev() * rob_motors[i]).log() * 0.5).unit().rev())
#         lines_b.append(((obj_motors[j] * obj_motors[i].rev()).log() * 0.5).unit())

In [259]:
n_lines = len(a_trfs)
motors_a = [vsr.Vec(*pos).trs() * vsr.Rot(vsr.Vec(*axis).unduale() * angle / 2.)
              for pos, (axis, angle) in [(m3d.Transform(pose).pos, 
                                          m3d.Transform(pose).orient.axis_angle)
                                         for pose in a_trfs]]
motors_b = [vsr.Vec(*pos).trs() * vsr.Rot(vsr.Vec(*axis).unduale() * angle / 2.)
              for pos, (axis, angle) in [(m3d.Transform(pose).pos, 
                                          m3d.Transform(pose).orient.axis_angle)
                                         for pose in b_trfs]]
lines_a = [motor.log() for motor in motors_a]
lines_b = [motor.log() for motor in motors_b]

#### Estimate hand-eye motor

In [256]:
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(lines_a, lines_b):
    mes.add_line_correspondences_residual_block(b,a)
#     mes.add_line_angle_distance_residual_block(b,a)
mes.set_parameterization('POLAR_DECOMPOSITION')
mes.linear_solver_type = "DENSE_QR"
mes.max_num_iterations = 25
# mes.function_tolerance = 1e-12
estimated_motor, summary = mes.solve()
print(summary['brief_report'])

Ceres Solver Report: Iterations: 12, Initial cost: 8.488296e-01, Final cost: 1.251614e-04, Termination: CONVERGENCE


In [257]:
print(estimated_motor)

Mot: [ 0.0028 -0.0027 -0.68 -0.74 -0.053 -0.058 0.05 -0.018 ]


In [284]:
print(vsr.Drv(1,0,0).spin(estimated_motor))
print(vsr.Drv(0,1,0).spin(estimated_motor))
print(vsr.Drv(0,0,1).spin(estimated_motor))
print(vsr.Vec(0,0,0).null().spin(estimated_motor))

Drv: [ 0.088 -1 0.0079 ]
Drv: [ -1 -0.088 0.00053 ]
Drv: [ 0.00016 -0.0079 -1 ]
Pnt: [ 0.04172 0.09897 0.1578 1 0.01822 ]


In [285]:
#### Combination of estimates

In [287]:
bivectors = [np.array((r * estimated_motor * o.rev()).log()) 
             for r, o in zip(rob_motors, obj_motors)]
m = vsr.Dll(*np.average(bivectors,0)).exp()

In [311]:
print(vsr.Drv(1,0,0).spin(m))
print(vsr.Drv(0,1,0).spin(m))
print(vsr.Drv(0,0,1).spin(m))
print(vsr.Vec(0,0,0).null().spin(m))

Drv: [ 0.72 -0.69 0.046 ]
Drv: [ 0.025 -0.041 -1 ]
Drv: [ 0.69 0.72 -0.013 ]
Pnt: [ -0.9711 -0.8614 0.8368 1 1.193 ]


In [301]:
def daniilidis(LAs, LBs):
    def skew(v):
        skv = roll(roll(diag(v.flatten()), 1, 1), -1, 0)
        return skv - skv.T
    
    Ds = []
    for LA, LB in zip(LAs, LBs):
        LA = LA.unit()
        LB = LB.unit()
        D = np.zeros((6,8))
        a = np.array(LA).copy()[:3]
        a = np.array([a[2], -a[1], a[0]])
        b = np.array(LB).copy()[:3]
        b = np.array([b[2], -b[1], b[0]])
        aprime = np.array(LA).copy()[3:]
        bprime = np.array(LB).copy()[3:]
        # Upper 3
        D[:3,0] = a - b
        D[:3,1:4] = skew(a + b)
        # Lower 3
        D[3:,0] = aprime - bprime
        D[3:,1:4] = skew(aprime + bprime)
        D[3:,4] = a - b 
        D[3:,5:8] = skew(a + b)  
        Ds.append(D)
    Ds = np.array(Ds).reshape(-1,8)
    
    [U, s, Vt] = np.linalg.svd(Ds)
    v7 = Vt.T[:,-2]
    v8 = Vt.T[:,-1]
    u1 = v7[:4]
    v1 = v7[4:]
    u2 = v8[:4]
    v2 = v8[4:]
    
    a = np.inner(u1,v1)
    b = np.inner(u1,v2) + np.inner(u2,v1)
    c = np.inner(u2,v2)
    [s1, s2] = np.roots([a,b,c])
    
    val1 = (s1**2 * np.inner(u1,u1)) + (2 * s1 * np.inner(u1,u2)) + (np.inner(u2,u2))
    val2 = (s2**2 * np.inner(u1,u1)) + (2 * s2 * np.inner(u1,u2)) + (np.inner(u2,u2))

    if val1 > val2:
        s = s1
        val = val1
    else:
        s = s2
        val = val2

    lambda2 = np.sqrt(1./val)
    lambda1 = s * lambda2
    
    m_arr = lambda1 * v7 + lambda2 * v8
    m_arr[1:4] = np.array([m_arr[3], -m_arr[2], m_arr[1]])
    m_arr[4:8] = np.roll(m_arr[4:8], -1)
    return vsr.Mot(*m_arr)

In [302]:
daniilidis_motor = daniilidis(lines_b, lines_a)

In [303]:
print(vsr.Drv(1,0,0).spin(daniilidis_motor.rev()))
print(vsr.Drv(0,1,0).spin(daniilidis_motor.rev()))
print(vsr.Drv(0,0,1).spin(daniilidis_motor.rev()))
print(vsr.Vec(0,0,0).null().spin(daniilidis_motor.rev()))

Drv: [ 0.13 -0.99 -0.013 ]
Drv: [ -0.99 -0.13 -0.03 ]
Drv: [ 0.028 0.017 -1 ]
Pnt: [ 0.06934 0.1201 0.1412 1 0.01959 ]


In [304]:
print(estimated_motor)
print(daniilidis_motor)

Mot: [ 0.0028 -0.0027 -0.68 -0.74 -0.053 -0.058 0.05 -0.018 ]
Mot: [ -0.015 0.005 0.66 0.75 0.046 0.052 -0.069 -0.013 ]


In [305]:
bivectors = [np.array((r * daniilidis_motor * o.rev()).log()) 
             for r, o in zip(rob_motors, obj_motors)]
m = vsr.Dll(*np.average(bivectors,0)).exp()
print(vsr.Drv(1,0,0).spin(m))
print(vsr.Drv(0,1,0).spin(m))
print(vsr.Drv(0,0,1).spin(m))
print(vsr.Vec(0,0,0).null().spin(m))

Drv: [ 0.72 -0.69 0.046 ]
Drv: [ 0.025 -0.041 -1 ]
Drv: [ 0.69 0.72 -0.013 ]
Pnt: [ -0.9711 -0.8614 0.8368 1 1.193 ]
